**Computer Vision-Object Tracking-SORT & YOLO Algorithms**

Second: Tracking by SORT

In [2]:
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=db8afa69374888476106197ad9b88ae2b54de181691a187abbff878d84ab96e4
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [3]:
!git clone https://github.com/AmirMansurian/SORT
%cd SORT/
!wget https://pjreddie.com/media/files/yolov3.weights -O config/yolov3.weights

Cloning into 'SORT'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 27 (delta 3), reused 22 (delta 0), pack-reused 0
Receiving objects: 100% (27/27), 10.76 MiB | 16.30 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/SORT
--2024-07-10 12:48:58--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘config/yolov3.weights’

config/yolov3.weigh 100%[===================>] 236.52M  40.3MB/s    in 5.7s    

2024-07-10 12:49:04 (41.1 MB/s) - ‘config/yolov3.weights’ saved [248007048/248007048]



In [4]:
from models import *
from utils import *
import imageio
import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

import sys
sys.path.append('/content/SORT')

from sort import *
from sort import iou

import cv2
from IPython.display import clear_output

/content/SORT/sort.py:28: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def iou(bb_test,bb_gt):


Object Detection

In [5]:
config_path='config/yolov3.cfg'
weights_path='config/yolov3.weights'
class_path='config/coco.names'
img_size=416
conf_thres=0.8
nms_thres=0.4

# Load model and weights
model = Darknet(config_path, img_size=img_size) # Definition of detection model
model.load_weights(weights_path) # Loading network weights
model.cuda() # Using GPU
model.eval() # The model is already trained and only needs to be in evaluation mode
classes = utils.load_classes(class_path)
Tensor = torch.cuda.FloatTensor

/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [6]:
def detect_image(img):
    # Basic pre-processing on the image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])

    # Image to tensor conversion for network input
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))

   # Using the model in evaluation mode and giving an image to the trained model
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)

    return detections[0] # The output of the bounding boxes will be inside the image

In [7]:
def convertMillis(millseconds):
    seconds, millseconds = divmod(millseconds, 1000)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    day, hours = divmod(hours, 24)
    seconds = int(seconds + millseconds/10000)
    return f"{int(hours)}:{int(minutes)}:{int(seconds)}"

Tracking

In [8]:
pip install lap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628945 sha256=e1fb256e6dce0760bb67192d8204dc4c4b91636034786563ebe603f70a0791cb
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built lap


In [ ]:
videopath = '/content/SORT/video/traffic_1.gif'

%pylab inline

cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

# Loading video
cap = cv2.VideoCapture('/content/SORT/video/traffic_1.gif')


mot_tracker = Sort() # Tracking model definition
Frames = [] # To save the final video

num_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


for ii in range(num_frame):
    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)
    time_report = convertMillis(timestamp)
    ret, frame = cap.read() # Loading a frame of video
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pilimg = Image.fromarray(frame)
    detections = detect_image(pilimg) # Giving the loaded frame to the detection network

    img = np.array(pilimg)
    pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
    unpad_h = img_size - pad_y
    unpad_w = img_size - pad_x

    if detections is not None:
        tracked_objects = mot_tracker.update(detections.cpu()) # Sending the detection output to the tracking network

        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)

        ########## Displaying the bounding boxes of objects and their ID on each frame ###########
        for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
            box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
            box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
            y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
            x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])

            color = colors[int(obj_id) % len(colors)]
            color = [i * 255 for i in color]
            cls = classes[int(cls_pred)]
            cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 1) # The bounding box around the object
            cv2.rectangle(frame, (x1, y1-20), (x1+len(cls)*15, y1), color, -1) # A bounding box above each object to display the class type and object ID

            cv2.putText(frame, cls + "-" + str(int(obj_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255,255,255), 1) # Text related to class name and ID

    Frames.append(frame) # Save processed frames

# Save the tracked video frames as a GIF
output_path = 'tracked_video_ZEYNAB.gif'
imageio.mimsave(output_path, Frames, fps=10)
